<a href="https://colab.research.google.com/github/Ramanarayanan/DeepLearningAndGenerativeAi/blob/main/Vector_database_and_Lama_model_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install transformers faiss-cpu datasets langchain langchain_community


In [10]:
!pip install langchain_huggingface

In [11]:
import os
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import Dataset
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFacePipeline
from langchain.chains import ConversationalRetrievalChain


In [20]:
from langchain.schema import Document

In [32]:
# Load custom dataset
data = [
    {"question": "What is SmartHome 360?", "answer": "SmartHome 360 is a smart home automation system that allows users to control their lights, security cameras, and thermostats remotely via a mobile app."},
    {"question": "How do I set up SmartHome 360?", "answer": "To set up SmartHome 360, download the app, create an account, and follow the in-app instructions to pair your devices."},
    {"question": "Can I control the lights remotely?", "answer": "Yes, SmartHome 360 allows you to control the lights remotely through the mobile app."},
    {"question": "Does SmartHome 360 support voice assistants?", "answer": "Yes, SmartHome 360 is compatible with Amazon Alexa, Google Assistant, and Apple HomeKit."},
    {"question": "What security features does SmartHome 360 offer?", "answer": "SmartHome 360 provides motion detection, real-time alerts, and remote monitoring through security cameras."},
    {"question": "Is SmartHome 360 easy to install?", "answer": "Yes, SmartHome 360 is designed to be easy to install with step-by-step guides and customer support available."},
    {"question": "How long does the battery last?", "answer": "The battery life of SmartHome 360 devices varies, but on average, they last up to 6 months on a single charge."},
     {
        "question": "What is TDMS",
        "answer": "The Therapy Data Management System – TDMS – is seamlessly integrated in existing working processes and supports the user in his or her daily tasks, such as ultra-filtration management, prescription transfer to dialysis devices or treatment documentation. As a total integrated solution, the Therapy Data Management System is a seamless link between the world of dialysis and information technology and provides the right tool for every professional in a dialysis center."
    },
    {
        "question": "What is Therapy Support Suite or TSS",
        "answer": "At a glance The Therapy Support Suite (TSS) is the central clinical management software within TDMS. As such it supports the user in carrying out, creating and managing patient prescriptions, medication plans, documentation of treatment data, and patient-related laboratory data. Furthermore, it provides enhanced reporting capabilities and allows the professional management of an association of dialysis centers."
    },
    {
        "question": "What is Therapy Monitor (TMon)",
        "answer": "Monitoring software for the Therapy Data Management System.The Therapy Monitor is our solution for online documentation of various data collected before, during and after dialysis. The innovative bed-side monitoring is designed for advanced documentation on the 5008 CorDiax and 5008S CorDiax and thus directly at the bed-side of the patient"
    },
    {
        "question": "What is communication Data Link (cDL)",
        "answer": "Interfacing TDMS with external hospital information systems (HIS)"
    },
    {
        "question": "PatientOnLine (POL)",
        "answer": "PD management software designed for your teamIndividual prescription support,Treatment results & analysis,Patient follow up by in-depth overview of PD therapy"
    },
    {
        "question": "About TMON",
        "answer": "The TMon Study was performed in a dialysis unit of a hospital in Saarbrücken, Germany. There, the dialysis workflow of the treatments of 12 patients was analyzed for two nurses before and after the implementation of the Therapy Monitor. The study followed two approaches in order to identify the benefits of TMon for its users and the patients. The measurement of all dialysis related process times allowed for a quantitative analysis, whereas interviews with doctors and nurses formed the qualitative part of the study,"
    },
    {
        "question": "Study results on TMON",
        "answer": "With the introduction of Therapy Monitor a change of the dialysis workflow becomes necessary. This improvement leads to savings of nursing time spent during dialysis. Overall, the workflow optimization and speed up of the processes results in a saving of approximately 25 minutes or 40% per dialysis treatment. This finally leads to more patient comfort and less stress for nursing staff."
    }
]

# Convert to Dataset
dataset = Dataset.from_dict({"question": [entry["question"] for entry in data], "answer": [entry["answer"] for entry in data]})

# Convert each question-answer pair into a Document object
documents = [Document(page_content=entry["question"], metadata={"answer": entry["answer"]}) for entry in data]

In [33]:
from langchain_huggingface import HuggingFacePipeline

# llm = HuggingFacePipeline.from_model_id(
#     model_id="microsoft/Phi-3-mini-4k-instruct",
#     task="text-generation",
#     pipeline_kwargs={
#         "max_new_tokens": 100,
#         "top_k": 50,
#         "temperature": 0.1,
#     },
# )
# llm.invoke("Hugging Face is")


# # Initialize HuggingFace Embedding Model
# embedding_model = HuggingFaceEmbeddings(model_name="microsoft/Phi-3-mini-4k-instruct")

# # Create FAISS vector store
# faiss_store = FAISS.from_documents([entry["question"] for entry in data], embedding_model)

# # Save FAISS index
# faiss_store.save_local("faiss_index")


In [34]:
!pip install langchain_groq

In [35]:
from langchain_groq import ChatGroq

In [36]:
from google.colab import userdata
userdata.get('groq_api_key_valu')
print(userdata.get('groq_api_key_valu'))

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


In [37]:
from langchain.embeddings import HuggingFaceEmbeddings

In [38]:
llm = ChatGroq(
    temperature=0,
    groq_api_key=userdata.get('groq_api_key_valu'),
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("Show nifty index as on today ...")

# Initialize HuggingFace Embedding Model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
documents = [Document(page_content=entry["question"], metadata={"answer": entry["answer"]}) for entry in data]
# Create FAISS vector store
faiss_store = FAISS.from_documents(documents, embedding_model)

# Save FAISS index
faiss_store.save_local("faiss_index")


In [39]:
# Use HuggingFace LLM (like GPT-3 or GPT-4)


# Create a ConversationalRetrievalChain
qa_chain = ConversationalRetrievalChain.from_llm(llm, faiss_store.as_retriever())


In [40]:
def chat_with_bot(user_input, chat_history=[]): # add chat_history
    response = qa_chain({"question": user_input, "chat_history": chat_history}) # Include chat_history in the input
    chat_history.append((user_input, response['answer']))  # Update chat history with the current turn
    return response, chat_history # Return both response and chat_history

# Test the chatbot
chat_history = [] # Initialize chat_history
response, chat_history = chat_with_bot("What is Therapy Support Suite and Therapy Monitor ?", chat_history) # Pass chat_history to chat_with_bot
print(response)

response, chat_history = chat_with_bot("Explain about PatientOnLine (POL)?", chat_history) # Pass the updated chat_history
print(response)

{'question': 'What is Therapy Support Suite and Therapy Monitor ?', 'chat_history': [('What is Therapy Support Suite and Therapy Monitor ?', "Therapy Support Suite (TSS) and Therapy Monitor (TMon) appear to be related to healthcare or therapy services, but I don't have enough information to provide a detailed explanation of what they are or how they work. I can only confirm that they seem to be related to therapy support, but I don't know their specific functions or purposes.")], 'answer': "Therapy Support Suite (TSS) and Therapy Monitor (TMon) appear to be related to healthcare or therapy services, but I don't have enough information to provide a detailed explanation of what they are or how they work. I can only confirm that they seem to be related to therapy support, but I don't know their specific functions or purposes."}
{'question': 'Explain about PatientOnLine (POL)?', 'chat_history': [('What is Therapy Support Suite and Therapy Monitor ?', "Therapy Support Suite (TSS) and Therap